In [1]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [2]:
# Initialize a network
# each node is implemented using dictionary
# each layer is a list of nodes, and network is a list of layers
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [3]:
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)


[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [4]:
# Calculate neuron activation for an input
# basically weighted sum of inputs to the neuron
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


In [5]:
# Transfer neuron activation
# sigmoid function
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))


In [6]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = [] # store input for next layer/ output of this layer

        # traverse through each node and calculate its output
		for neuron in layer: # each neuron is a dictionary
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation) # storing output in the dictionary/neuron
			new_inputs.append(neuron['output'])

		inputs = new_inputs

	return inputs

In [7]:
# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None] # None represents the bias input, it can be deleted 
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [8]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)


In [9]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
            # for hidden layers
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
            # for output layer
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [10]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [11]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1] # excluding the target value
		if i != 0:
            # for layers other than first hidden layer take input from previous layer
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
                # update each weight for the neuron
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            # updating the bias weight
			neuron['weights'][-1] += l_rate * neuron['delta']

In [12]:
# Train a network for a fixed number of epochs
# epoch refers to one full iteration of the dataset
# l_rate is the learning rate
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row) # get output using current weights
            
            # since the target/class value is known, probability for that value is 1 and rest is 0
			expected = [0 for i in range(n_outputs)] # initialize to zero
			expected[row[-1]] = 1 # set the class value to 1
            
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
        
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [13]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights

In [14]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [15]:
# Test making predictions with the network
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]}, {'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
	[{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights': [-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


In [16]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [17]:
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

In [18]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [19]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [20]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [21]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [22]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [23]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'Crop_recommendation.csv'
dataset = load_csv(filename)
dataset.pop(0)

target_set = set(row[-1] for row in dataset)
target_index = {}
i = 0
for target in target_set :
    target_index[target] = i
    i+=1
    
print(target_index)

# replace class labels (strings) with integer equivalent (index)
for row in dataset:
    for i in range(len(row)-1):
        row[i] = float(row[i])
    row[-1] = int(target_index[row[-1]])

# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 10
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

{'apple': 0, 'kidneybeans': 1, 'orange': 2, 'muskmelon': 3, 'coffee': 4, 'papaya': 5, 'lentil': 6, 'banana': 7, 'watermelon': 8, 'coconut': 9, 'chickpea': 10, 'mungbean': 11, 'pomegranate': 12, 'grapes': 13, 'rice': 14, 'pigeonpeas': 15, 'mothbeans': 16, 'mango': 17, 'blackgram': 18, 'cotton': 19, 'maize': 20, 'jute': 21}
>epoch=0, lrate=0.300, error=2738.511
>epoch=1, lrate=0.300, error=1680.449
>epoch=2, lrate=0.300, error=1676.237
>epoch=3, lrate=0.300, error=1667.920
>epoch=4, lrate=0.300, error=1646.148
>epoch=5, lrate=0.300, error=1601.124
>epoch=6, lrate=0.300, error=1563.435
>epoch=7, lrate=0.300, error=1511.108
>epoch=8, lrate=0.300, error=1420.388
>epoch=9, lrate=0.300, error=1313.600
>epoch=10, lrate=0.300, error=1218.365
>epoch=11, lrate=0.300, error=1137.566
>epoch=12, lrate=0.300, error=1067.227
>epoch=13, lrate=0.300, error=1001.652
>epoch=14, lrate=0.300, error=939.016
>epoch=15, lrate=0.300, error=881.411
>epoch=16, lrate=0.300, error=829.074
>epoch=17, lrate=0.300, er

>epoch=206, lrate=0.300, error=83.847
>epoch=207, lrate=0.300, error=83.618
>epoch=208, lrate=0.300, error=83.390
>epoch=209, lrate=0.300, error=83.165
>epoch=210, lrate=0.300, error=82.942
>epoch=211, lrate=0.300, error=82.720
>epoch=212, lrate=0.300, error=82.501
>epoch=213, lrate=0.300, error=82.283
>epoch=214, lrate=0.300, error=82.068
>epoch=215, lrate=0.300, error=81.854
>epoch=216, lrate=0.300, error=81.642
>epoch=217, lrate=0.300, error=81.431
>epoch=218, lrate=0.300, error=81.223
>epoch=219, lrate=0.300, error=81.016
>epoch=220, lrate=0.300, error=80.811
>epoch=221, lrate=0.300, error=80.607
>epoch=222, lrate=0.300, error=80.406
>epoch=223, lrate=0.300, error=80.206
>epoch=224, lrate=0.300, error=80.007
>epoch=225, lrate=0.300, error=79.810
>epoch=226, lrate=0.300, error=79.615
>epoch=227, lrate=0.300, error=79.421
>epoch=228, lrate=0.300, error=79.228
>epoch=229, lrate=0.300, error=79.038
>epoch=230, lrate=0.300, error=78.848
>epoch=231, lrate=0.300, error=78.660
>epoch=232, 

>epoch=422, lrate=0.300, error=56.169
>epoch=423, lrate=0.300, error=56.075
>epoch=424, lrate=0.300, error=55.981
>epoch=425, lrate=0.300, error=55.885
>epoch=426, lrate=0.300, error=55.789
>epoch=427, lrate=0.300, error=55.692
>epoch=428, lrate=0.300, error=55.594
>epoch=429, lrate=0.300, error=55.495
>epoch=430, lrate=0.300, error=55.395
>epoch=431, lrate=0.300, error=55.293
>epoch=432, lrate=0.300, error=55.191
>epoch=433, lrate=0.300, error=55.088
>epoch=434, lrate=0.300, error=54.984
>epoch=435, lrate=0.300, error=54.878
>epoch=436, lrate=0.300, error=54.772
>epoch=437, lrate=0.300, error=54.664
>epoch=438, lrate=0.300, error=54.556
>epoch=439, lrate=0.300, error=54.446
>epoch=440, lrate=0.300, error=54.335
>epoch=441, lrate=0.300, error=54.223
>epoch=442, lrate=0.300, error=54.111
>epoch=443, lrate=0.300, error=53.997
>epoch=444, lrate=0.300, error=53.882
>epoch=445, lrate=0.300, error=53.766
>epoch=446, lrate=0.300, error=53.650
>epoch=447, lrate=0.300, error=53.532
>epoch=448, 

>epoch=137, lrate=0.300, error=103.475
>epoch=138, lrate=0.300, error=102.937
>epoch=139, lrate=0.300, error=102.408
>epoch=140, lrate=0.300, error=101.888
>epoch=141, lrate=0.300, error=101.377
>epoch=142, lrate=0.300, error=100.874
>epoch=143, lrate=0.300, error=100.378
>epoch=144, lrate=0.300, error=99.891
>epoch=145, lrate=0.300, error=99.411
>epoch=146, lrate=0.300, error=98.939
>epoch=147, lrate=0.300, error=98.474
>epoch=148, lrate=0.300, error=98.016
>epoch=149, lrate=0.300, error=97.565
>epoch=150, lrate=0.300, error=97.120
>epoch=151, lrate=0.300, error=96.682
>epoch=152, lrate=0.300, error=96.250
>epoch=153, lrate=0.300, error=95.825
>epoch=154, lrate=0.300, error=95.406
>epoch=155, lrate=0.300, error=94.992
>epoch=156, lrate=0.300, error=94.585
>epoch=157, lrate=0.300, error=94.183
>epoch=158, lrate=0.300, error=93.786
>epoch=159, lrate=0.300, error=93.395
>epoch=160, lrate=0.300, error=93.009
>epoch=161, lrate=0.300, error=92.628
>epoch=162, lrate=0.300, error=92.252
>epoc

>epoch=353, lrate=0.300, error=56.896
>epoch=354, lrate=0.300, error=56.809
>epoch=355, lrate=0.300, error=56.722
>epoch=356, lrate=0.300, error=56.635
>epoch=357, lrate=0.300, error=56.549
>epoch=358, lrate=0.300, error=56.464
>epoch=359, lrate=0.300, error=56.378
>epoch=360, lrate=0.300, error=56.294
>epoch=361, lrate=0.300, error=56.210
>epoch=362, lrate=0.300, error=56.126
>epoch=363, lrate=0.300, error=56.043
>epoch=364, lrate=0.300, error=55.960
>epoch=365, lrate=0.300, error=55.878
>epoch=366, lrate=0.300, error=55.796
>epoch=367, lrate=0.300, error=55.715
>epoch=368, lrate=0.300, error=55.634
>epoch=369, lrate=0.300, error=55.554
>epoch=370, lrate=0.300, error=55.474
>epoch=371, lrate=0.300, error=55.395
>epoch=372, lrate=0.300, error=55.316
>epoch=373, lrate=0.300, error=55.238
>epoch=374, lrate=0.300, error=55.160
>epoch=375, lrate=0.300, error=55.083
>epoch=376, lrate=0.300, error=55.005
>epoch=377, lrate=0.300, error=54.928
>epoch=378, lrate=0.300, error=54.851
>epoch=379, 

...

>epoch=1, lrate=0.300, error=1677.613
>epoch=2, lrate=0.300, error=1661.638
>epoch=3, lrate=0.300, error=1624.029
>epoch=4, lrate=0.300, error=1603.461
>epoch=5, lrate=0.300, error=1585.797
>epoch=6, lrate=0.300, error=1548.264
>epoch=7, lrate=0.300, error=1493.906
>epoch=8, lrate=0.300, error=1438.653
>epoch=9, lrate=0.300, error=1375.666
>epoch=10, lrate=0.300, error=1293.507
>epoch=11, lrate=0.300, error=1204.417
>epoch=12, lrate=0.300, error=1126.316
>epoch=13, lrate=0.300, error=1055.099
>epoch=14, lrate=0.300, error=984.016
>epoch=15, lrate=0.300, error=911.773
>epoch=16, lrate=0.300, error=841.425
>epoch=17, lrate=0.300, error=775.735
>epoch=18, lrate=0.300, error=716.241
>epoch=19, lrate=0.300, error=663.397
>epoch=20, lrate=0.300, error=616.902
>epoch=21, lrate=0.300, error=576.447
>epoch=22, lrate=0.300, error=541.440
>epoch=23, lrate=0.300, error=510.900
>epoch=24, lrate=0.300, error=483.822
>epoch=25, lrate=0.300, error=459.399
>epoch=26, lrate=0.300, error=437.048
>epoch=2

>epoch=217, lrate=0.300, error=55.730
>epoch=218, lrate=0.300, error=55.574
>epoch=219, lrate=0.300, error=55.420
>epoch=220, lrate=0.300, error=55.267
>epoch=221, lrate=0.300, error=55.115
>epoch=222, lrate=0.300, error=54.964
>epoch=223, lrate=0.300, error=54.815
>epoch=224, lrate=0.300, error=54.668
>epoch=225, lrate=0.300, error=54.521
>epoch=226, lrate=0.300, error=54.376
>epoch=227, lrate=0.300, error=54.233
>epoch=228, lrate=0.300, error=54.090
>epoch=229, lrate=0.300, error=53.949
>epoch=230, lrate=0.300, error=53.808
>epoch=231, lrate=0.300, error=53.670
>epoch=232, lrate=0.300, error=53.532
>epoch=233, lrate=0.300, error=53.395
>epoch=234, lrate=0.300, error=53.260
>epoch=235, lrate=0.300, error=53.125
>epoch=236, lrate=0.300, error=52.992
>epoch=237, lrate=0.300, error=52.860
>epoch=238, lrate=0.300, error=52.728
>epoch=239, lrate=0.300, error=52.598
>epoch=240, lrate=0.300, error=52.469
>epoch=241, lrate=0.300, error=52.341
>epoch=242, lrate=0.300, error=52.214
>epoch=243, 

>epoch=433, lrate=0.300, error=37.746
>epoch=434, lrate=0.300, error=37.702
>epoch=435, lrate=0.300, error=37.658
>epoch=436, lrate=0.300, error=37.614
>epoch=437, lrate=0.300, error=37.570
>epoch=438, lrate=0.300, error=37.527
>epoch=439, lrate=0.300, error=37.484
>epoch=440, lrate=0.300, error=37.441
>epoch=441, lrate=0.300, error=37.398
>epoch=442, lrate=0.300, error=37.355
>epoch=443, lrate=0.300, error=37.313
>epoch=444, lrate=0.300, error=37.271
>epoch=445, lrate=0.300, error=37.229
>epoch=446, lrate=0.300, error=37.187
>epoch=447, lrate=0.300, error=37.146
>epoch=448, lrate=0.300, error=37.104
>epoch=449, lrate=0.300, error=37.063
>epoch=450, lrate=0.300, error=37.022
>epoch=451, lrate=0.300, error=36.981
>epoch=452, lrate=0.300, error=36.941
>epoch=453, lrate=0.300, error=36.900
>epoch=454, lrate=0.300, error=36.860
>epoch=455, lrate=0.300, error=36.820
>epoch=456, lrate=0.300, error=36.780
>epoch=457, lrate=0.300, error=36.741
>epoch=458, lrate=0.300, error=36.701
>epoch=459, 

>epoch=148, lrate=0.300, error=97.524
>epoch=149, lrate=0.300, error=97.145
>epoch=150, lrate=0.300, error=96.770
>epoch=151, lrate=0.300, error=96.400
>epoch=152, lrate=0.300, error=96.034
>epoch=153, lrate=0.300, error=95.673
>epoch=154, lrate=0.300, error=95.317
>epoch=155, lrate=0.300, error=94.966
>epoch=156, lrate=0.300, error=94.618
>epoch=157, lrate=0.300, error=94.276
>epoch=158, lrate=0.300, error=93.937
>epoch=159, lrate=0.300, error=93.603
>epoch=160, lrate=0.300, error=93.273
>epoch=161, lrate=0.300, error=92.946
>epoch=162, lrate=0.300, error=92.624
>epoch=163, lrate=0.300, error=92.306
>epoch=164, lrate=0.300, error=91.992
>epoch=165, lrate=0.300, error=91.681
>epoch=166, lrate=0.300, error=91.374
>epoch=167, lrate=0.300, error=91.071
>epoch=168, lrate=0.300, error=90.772
>epoch=169, lrate=0.300, error=90.476
>epoch=170, lrate=0.300, error=90.183
>epoch=171, lrate=0.300, error=89.894
>epoch=172, lrate=0.300, error=89.608
>epoch=173, lrate=0.300, error=89.325
>epoch=174, 

>epoch=364, lrate=0.300, error=61.178
>epoch=365, lrate=0.300, error=61.094
>epoch=366, lrate=0.300, error=61.011
>epoch=367, lrate=0.300, error=60.928
>epoch=368, lrate=0.300, error=60.846
>epoch=369, lrate=0.300, error=60.763
>epoch=370, lrate=0.300, error=60.681
>epoch=371, lrate=0.300, error=60.600
>epoch=372, lrate=0.300, error=60.519
>epoch=373, lrate=0.300, error=60.438
>epoch=374, lrate=0.300, error=60.357
>epoch=375, lrate=0.300, error=60.276
>epoch=376, lrate=0.300, error=60.196
>epoch=377, lrate=0.300, error=60.117
>epoch=378, lrate=0.300, error=60.037
>epoch=379, lrate=0.300, error=59.958
>epoch=380, lrate=0.300, error=59.879
>epoch=381, lrate=0.300, error=59.801
>epoch=382, lrate=0.300, error=59.722
>epoch=383, lrate=0.300, error=59.645
>epoch=384, lrate=0.300, error=59.567
>epoch=385, lrate=0.300, error=59.490
>epoch=386, lrate=0.300, error=59.413
>epoch=387, lrate=0.300, error=59.336
>epoch=388, lrate=0.300, error=59.259
>epoch=389, lrate=0.300, error=59.183
>epoch=390, 